### Info electrodes, rois, labels, AUC SCORE
#### From the beginning - electrodes included in the analysis
    Create the big file data 'all_subjects_info_elecs.csv'

In [ ]:
from brainpipe.system import study
from os.path import isfile, join
import numpy as np
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib notebook

conds = ['low','high']
###############################################################################
st = study('Olfacto')
path_npz = join(st.path, 'figure/1_Classif_PAC_E_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'/')
f_form = '{}_sources_{}_odor_'+conds[0]+'_'+conds[1]+'_sel_physFT.npz'
f_form = join(path_npz, f_form)
###############################################################################

freqs = ['0_VLFC', '1_delta', '2_theta', '3_alpha']

mat = np.load(f_form.format('All_subjects','2_theta'))
print(mat.files)
subjects = mat['su_codes'][:,np.newaxis]
s_elec, s_channels = mat['s_elec'][:,np.newaxis], mat['s_channels'][:,np.newaxis]
s_aal, s_BA = mat['s_aal'][:,np.newaxis], mat['s_BA'][:,np.newaxis]
aal_RL = np.array([s_aal[i][0][:-4][:] for i in range(s_aal.shape[0])])[:,np.newaxis]
s_Mai_RL = mat['s_MAI_RL'][:,np.newaxis]
print(subjects.shape, s_elec.shape, s_channels.shape, s_aal.shape,aal_RL.shape,s_BA.shape,s_Mai_RL.shape)
elec_data = np.concatenate((subjects, s_elec, s_channels,s_aal,aal_RL,s_BA,s_Mai_RL), axis=1)
df0 = pd.DataFrame(elec_data, columns=['subjects','elecs','channels','aal',
                                       'aal_RL','BA','s_Mai_RL'])
#cols = pd.MultiIndex.from_product([['subjects','elecs','channels','labels','aal','aal_RL'],['_']])
#df = pd.DataFrame(elec_data, columns=cols)
dfname = '0_all_subjects_info_elecs.csv'
df0.to_csv(path_npz+dfname)

# load all electrodes information and create a df
idx_all, feat0_all, feat1_all, da_all = np.array([]),np.array([]),np.array([]),np.array([])
for freq in freqs:
    mat = np.load(f_form.format('All_subjects',freq))
    n_elecs = mat['s_da'].shape[0]
    idx_elecs, feat0_elecs, feat1_elecs, da_elecs = np.array([]),np.array([]),np.array([]),np.array([])
    for elec in range(n_elecs):
        da_elec = mat['s_da'][elec]
        feat0_elec = np.mean(mat['s_elec_feat0'][elec])
        feat1_elec = np.mean(mat['s_elec_feat1'][elec])
        feat0_elecs = np.hstack((feat0_elecs,feat0_elec)) if np.size(feat0_elecs) else feat0_elec
        feat1_elecs = np.hstack((feat1_elecs, feat1_elec)) if np.size(feat1_elecs) else feat1_elec
        da_elecs = np.hstack((da_elecs,da_elec)) if np.size(da_elecs) else da_elec
    feat0_all= np.vstack((feat0_all, feat0_elecs)) if np.size(feat0_all) else feat0_elecs
    feat1_all= np.vstack((feat1_all, feat1_elecs)) if np.size(feat1_all) else feat1_elecs
    da_all = np.vstack((da_all, da_elecs)) if np.size(da_all) else da_elecs
df2 = pd.DataFrame(da_all.T, columns=[freq+'_AUC' for freq in freqs])
df3 = pd.DataFrame(feat0_all.T, columns=[freq+'_pac0' for freq in freqs])
df4 = pd.DataFrame(feat1_all.T, columns=[freq+'_pac1' for freq in freqs])
df_all = pd.concat((df0,df2,df3,df4),axis=1)
#df_all = df.join(df2, how='left')
df_all_name = '1_all_subjects_info_elecs_AUC_pac.csv'
df_all.to_csv(path_npz+df_all_name)

#### Plot nb of signif electrodes by subjects and freq - TOTAL

In [ ]:
from brainpipe.system import study
from os import makedirs
from os.path import isfile, join, exists
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
%matplotlib notebook
from itertools import product

conds,ths = ['low', 'high'],['05']
min_wins = ['1']
###############################################################################
st = study('Olfacto')
path = join(st.path, 'figure/1_Classif_PAC_E_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'/')
path_mask = path+'masks_stat/'
path2save = path+'Signif_elecs'+ths[0]+'/'
freqs = ['0_VLFC','1_delta', '2_theta', '3_alpha']#['0_VLFC','1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
freqnames = [freq[2:] for freq in freqs]
minwin_names = ['min_win'+str(min_win[:]) for min_win in min_wins]
f_form = 'All_subjects_mask_stat_{}_minwin{}_th{}.npy'
f_form = join(path_mask, f_form)
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

all_mask=np.array([])
for th, freq in product(ths,freqs):
    for min_win in min_wins:
        mask = np.load(f_form.format(freq,min_win,th))[:,np.newaxis]
        all_mask = np.concatenate((all_mask,mask), axis=1) if all_mask.size else mask
#create a DataFrame with all stats for all freqs
cols = pd.MultiIndex.from_product([freqnames,minwin_names])
df = pd.DataFrame(all_mask, columns=cols) #boolean df
# Tranform True value by 0 and False value by 1
df = df*-1+1

# Load elecs file information
filename = '1_all_subjects_info_elecs_AUC_pac.csv'
df2 = pd.read_csv(path+filename, sep=',')
df2 = pd.concat([df2,df], axis=1)
df2.drop(['Unnamed: 0'],inplace=True, axis=1)
df2.set_index(['subjects'], inplace=True)
df_name = '2_all_subjects_info_elecs_AUC_Signif_th_'+th+'.csv'
df2.to_csv(path+df_name)

#Plot nb of significant electrodes by win
fig = plt.figure()
plt.title('Significant electrodes by frequency band')
plt.ylabel('Nb of electrodes')
xticks, w = np.arange(0,len(freqnames)*len(min_wins),len(min_wins)), 0.8
plt.xticks(xticks,freqnames)
#plt.ylim(bottom=0, top=300)

colors_w = ['gold','darkorange','orangered','red','brown']
for i, win in enumerate(minwin_names):
    df3 = df2.filter(like=(win)).agg(('sum','std','sem'))
    plt.bar(xticks,df3.loc[['sum']].values[0],color=colors_w[i],
            yerr=df3.loc[['std']].values[0], label=minwin_names[i])
plt.show()
plt.legend(loc=0,handletextpad=0.1, frameon=False)
plt.savefig(path2save+'Total_elec_signif_freq_4500_th_'+th+'.png')

#Plot nb of significant electrodes by win and subjects
colors = {'S0':'darkblue', 'S1':'royalblue', 'S2':'deepskyblue', 
        'S3':'mediumspringgreen','S4':'yellow', 'S5':'darkorange', 'S6':'red'}
df4 = df2.groupby(['subjects']).agg(('sum','std'))
xticks, w = np.arange(0,len(freqnames)*len(min_wins),len(min_wins)), 0.8
for i,su in enumerate(df4.index):
    fig = plt.figure()
    for j,min_win in enumerate(min_wins):
        #df4.iloc[[i]].filter(like='sum').filter(like=minwin_names[j]).values[0]
        count = df4.iloc[[i]].filter(like='sum').filter(like=minwin_names[j]).values[0]
        #err = df4.iloc[[i]].filter(like='std').filter(like=minwin_names[j]).values[0]
        print('shape to plot',count.shape, xticks.shape)
        plt.bar(xticks, count, color=colors[su], label=minwin_names[j])
    plt.title('Significant electrodes by frequency band for '+su)
    plt.ylabel('Nb of electrodes')
    plt.xticks(xticks,freqnames)
    #plt.ylim(bottom=0, top=(50))
    plt.legend(loc=0,handletextpad=0.1, frameon=False)
    plt.show()
    plt.savefig(path2save+su+'_elec_signif_freq_4500_th_'+th+'.png')


#### Plot signif electrodes by frequency bands, subjects and ROIs

In [2]:
from brainpipe.system import study
from os import makedirs
from os.path import isfile, join, exists
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from itertools import product

conds,th = ['low','high'],'05'
methods = ['s_Mai_RL']
min_wins, min_sigs = ['1'],[1]#[1,2,3,4,5]
###############################################################################
st = study('Olfacto')
path = join(st.path, 'figure/1_Classif_PAC_E_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'/')
path2save = path+'/Signif_ROIs'+th+'/'
freqs = ['0_VLFC','1_delta', '2_theta', '3_alpha']#['0_VLFC','1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
freqnames = [freq[2:] for freq in freqs]
minwin_names = ['min_win'+str(min_win[:]) for min_win in min_wins]
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################

for method in methods:
    filename = '2_all_subjects_info_elecs_AUC_Signif_th_'+th+'.csv'
    df = pd.read_csv(path+filename, sep=',')
    df = df.drop(['elecs'], axis=1).set_index(['subjects', method])
    df2 = df.groupby(['subjects',method]).sum().unstack()
    df2 = df2.fillna(0)
    df2 = df2.swaplevel(0,1,axis =1)
    #print(df2.filter(like='Hippocampus').filter(like='theta'))

    colors = {'S0':'darkblue', 'S1':'royalblue', 'S2':'deepskyblue', 
            'S3':'mediumspringgreen','S4':'yellow', 'S5':'darkorange', 'S6':'red'}

    for freq in freqnames:
        for min_win, min_sig in product(min_wins,min_sigs):
            df_sel = df2.filter(like='min_win'+min_win).filter(like=freq)
            df3=df_sel
            for i in range(len(df_sel.columns.levels[0])):
                df3 = df3
                col = df_sel.iloc[:,[i]]
                if df_sel.iloc[:,[i]].values.astype(bool).sum() < min_sig:
                    df3 = df3.drop(col,axis=1)
                else:
                    continue
            print('freq', freq, 'min_win',min_win, 'data signif',df3.shape)
            if df3.shape[1]>0:
                names = np.array(df3.columns.get_level_values(0).values)
                print(names)
                np.save((path2save+method+'_signif_min_'+str(min_sig)+'_'+freq+'_min_win'+min_win+'_th_'+th+'.npy'),names)
                bottom = np.zeros(len(df3.columns))
                for s in df3.index:
                    count = df3.iloc[df3.index == s].values[0]
                    xpts = range(len(count))
                    plt.bar(xpts, count, bottom=bottom, color=colors[s], label=s, width=0.8)
                    bottom += count
                plt.title('Significant electrodes by ROI/Subject for '+freq+' min_win'+min_win+' th '+th)
                plt.ylabel('Nb of electrodes')
                plt.xticks(xpts,list(df3.columns.get_level_values(0)),rotation=90)
                plt.tight_layout()
                plt.ylim(bottom=0, top=5)
                plt.xlim(xmin=-1,xmax=9)
                plt.legend(loc='best')
                plt.savefig(path2save+method+'_signif_min_'+str(min_sig)+'_'+freq+'_min_win'+min_win+'_th_'+th+'.png')
                plt.clf()
                plt.close()

-> Olfacto loaded
freq VLFC min_win 1 data signif (7, 5)
['HC' 'Ins' 'MFG' 'OFC' 'PHG']
freq delta min_win 1 data signif (7, 5)
['HC' 'IFG' 'MTG' 'OFC' 'STG']
freq theta min_win 1 data signif (7, 9)
['ACC' 'Amg-PirT' 'HC' 'IFG' 'ITG' 'MFG' 'MTG' 'SFG' 'STG']
freq alpha min_win 1 data signif (7, 6)
['ACC' 'FuG' 'Ins' 'MTG' 'OFC' 'SFG']


In [ ]:
from brainpipe.system import study
from os import makedirs
from os.path import isfile, join, exists
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from itertools import product

conds,th, bsl = ['low','high'],'01','None'
method = 's_Mai_RL'
min_wins, min_sigs = ['1'],[2]#[1,2,3,4,5]
###############################################################################
st = study('Olfacto')
path = join(st.path, 'figure/0_Classif_feater_E_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'_None/')
path2save = path+'/By_ROIs'+th+'/'
freqs = ['2_theta','6_gamma2']#['0_VLFC','1_delta', '2_theta', '3_alpha', '4_beta','5_gamma1','6_gamma2']
freqnames = [freq[2:] for freq in freqs]
minwin_names = ['min_win'+str(min_win[:]) for min_win in min_wins]
###############################################################################
if not exists(path2save):
    makedirs(path2save)
###############################################################################
colors = {'S0':'darkblue', 'S1':'royalblue', 'S2':'deepskyblue', 'S3':'mediumspringgreen',
           'S4':'yellow', 'S5':'darkorange', 'S6':'red'}
freq = '2_theta'
filename = '2_all_subjects_info_elecs_AUC_Signif_th_'+th+'_'+bsl+'.csv'
df = pd.read_csv(path+filename, sep=',')
df = df.rename(columns={"('gamma2', 'min_win1')": 'gamma2_sig', "('theta', 'min_win1')": 'theta_sig'})
df = df.loc[df[freq[2:]+'_sig'] == 1].set_index(['subjects', method])
df = df.filter(like='theta')
df['change_'+freq[2:]] = df[freq+'_feat1']-df[freq+'_feat0']
df = df.groupby(['s_Mai_RL','subjects']).agg(('count','mean','std','sem','max'))

for roi in df.index.levels[0]:
    print(roi)
    df_roi = df.xs(roi, level=0)
    df_roi_count = df_roi.filter(like='count').iloc[:,0]
    fig = plt.figure()
    fig
    #Plot nb of electrodes for each subjects
    bottom = np.zeros(1)
    for s in enumerate(df_roi_count.index):
        count = df_roi_count.iloc[df_roi_count.index.get_level_values('subjects') == s[1]].values[0]
        print(count)
        if count == 1:
            xpts = 1
        else:
            xpts = range(count.shape)
        plt.bar(xpts, count, bottom=bottom, color=colors[s[1]], label=s[1])
        bottom += count
    plt.show()
    0/0
    print(df_roi)

    

In [ ]:
df2 = df.groupby(['subjects',method]).sum().unstack()
    df2 = df2.fillna(0)
    df2 = df2.swaplevel(0,1,axis =1)
    #print(df2.filter(like='Hippocampus').filter(like='theta'))

    colors = {'S0':'darkblue', 'S1':'royalblue', 'S2':'deepskyblue', 
            'S3':'mediumspringgreen','S4':'yellow', 'S5':'darkorange', 'S6':'red'}

    for freq in freqnames:
        for min_win, min_sig in product(min_wins,min_sigs):
            df_sel = df2.filter(like='min_win'+min_win).filter(like=freq)
            df3=df_sel
            for i in range(len(df_sel.columns.levels[0])):
                df3 = df3
                col = df_sel.iloc[:,[i]]
                if df_sel.iloc[:,[i]].values.astype(bool).sum() < min_sig:
                    df3 = df3.drop(col,axis=1)
                else:
                    continue
            print('freq', freq, 'min_win',min_win, 'data signif',df3.shape)
            if df3.shape[1]>0:
                names = np.array(df3.columns.get_level_values(0).values)
                print(names)
                np.save((path2save+method+'_signif_min_'+str(min_sig)+'_'+freq+'_min_win'+min_win+'_th_'+th+'.npy'),names)
                bottom = np.zeros(len(df3.columns))
                for s in df3.index:
                    count = df3.iloc[df3.index == s].values[0]
                    xpts = range(len(count))
                    plt.bar(xpts, count, bottom=bottom, color=colors[s], label=s, width=0.8)
                    bottom += count
                plt.title('Significant electrodes by ROI/Subject for '+freq+' min_win'+min_win+' th '+th)
                plt.ylabel('Nb of electrodes')
                plt.xticks(xpts,list(df3.columns.get_level_values(0)),rotation=90)
                plt.tight_layout()
                #plt.ylim(bottom=0, top=60)
                plt.xlim(xmin=-1,xmax=20)
                plt.legend(loc='best')
                plt.savefig(path2save+method+'_signif_min_'+str(min_sig)+'_'+freq+'_min_win'+min_win+'_th_'+th+'.png')
                plt.clf()
                plt.close()